# 강남구 (1080)
- 작성자 : 황다은
- 작성일자 : 2020년 9월 6일
- 편집자 : 이준석
- 편집일자 : 2020년 9월 6일

In [1]:
from bs4 import BeautifulSoup
import datetime as dt
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

dat = dt.datetime.today().date() # 오늘 날짜

### 1. Raw File 불러오기

In [2]:
df = pd.read_csv("200905one_shot.csv")
soup = BeautifulSoup(df['html'][0], 'html.parser')

### 2. Regex Parsing 하기

In [3]:
# 너무 길어서 두 개로 나누었다. (가시적으로 표현하자.)
regex = 'tle\"\>\#(?P<num>.+?)\s확진자\\t\((?P<rsd>.+?)\).+?\/\s?(?P<dat>.+?)\<\/div.+?<li>감염경로\s\:\s?'
regex += '(?P<pth>.+?)\<\/li\>\s\<li\>(증상발현일\s\:\s)?(?P<sym>.+?)\<\/li>(.+?조치\s\:\s(?P<atn>.+?)\<\/li\>)?'

rows = soup.select('#tab-1 .acodList')
infos = [re.search(regex, str(r)) for r in rows]

### 3. 저장할 DataFrame 미리 만들기

In [4]:
pat_col = ['num', 'dat', 'pth', 'syy', 'syd', 'cnd', 'rsd', 'atn', 'dch', 'adt']
rou_col = ['num', 'ord', 'dat', 'rgl', 'rgt', 'frm', 'exd', 'msk', 'ste', 'mob']
patient = pd.DataFrame(columns = pat_col)
route = pd.DataFrame(columns = rou_col)

### 4. 귀찮은 전처리해주기

In [5]:
for i, inf in tqdm(enumerate(infos)):
    
    # regex 처리가 안 되는 애들은 어차피 정보가 없으므로 None 값임.
    if inf is not None:
        
        num = '1080' + '0' * (5-len(inf.group('num'))) + inf.group('num')
        syy = inf.group('sym') != '증상 없음'
        syd = inf.group('sym') * syy
        dch = ''; adt = '' # 정보가 없으면 이렇게 처리하자
        rsd, cnd, pth = inf.group('rsd'), inf.group('dat'), inf.group('pth')
        sym, atn = inf.group('sym'), inf.group('atn')
        new_pat = pd.DataFrame([(num, dat, pth, syy, syd, cnd, rsd, atn, dch, adt)], columns = pat_col)
        
        patient = pd.concat([patient, new_pat])
        
        new_route = pd.read_html(str(rows[i].select('table')[0]))[0].replace('※ 관내 동선 없음', '')
        new_route['rgl'] = new_route.지역 + new_route.위치
        new_route['frm'] = ''; new_route['msk'] = ''
        new_route['num'] = num
        new_route['ord'] = range(1, len(new_route)+1)
        new_route['dat'] = dat
        new_route = new_route.drop(['지역', '위치'], axis = 1)
        new_route.columns = ['rgt', 'exd', 'mob', 'ste', 'rgl', 'frm', 'msk', 'num', 'ord', 'dat']
        new_route = new_route.loc[:,rou_col]
        
        route = pd.concat([route, new_route])

138it [00:00, 309.29it/s]


### 5. 결과 잘 나왔나 확인하기

In [6]:
patient

,num,dat,pth,syy,syd,cnd,rsd,atn,dch,adt
0,108000207,2020-09-07,확진자 접촉,False,,2020-09-04,압구정동,None,,
0,108000206,2020-09-07,확진자 접촉,False,,2020-09-04,압구정동,None,,
0,108000205,2020-09-07,확진자 접촉,False,,2020-09-04,압구정동,None,,
0,108000204,2020-09-07,미상,True,8.30,2020-09-04,개포동,None,,
0,108000203,2020-09-07,확진자 접촉,True,8.24,2020-09-03,일원동,코이카생활치료센터 이송,,
0,108000202,2020-09-07,확진자 접촉,True,9.2,2020-09-03,논현동,코이카생활치료센터 이송,,
0,108000201,2020-09-07,확진자 접촉,True,9.2,2020-09-03,수서동,남산생활치료센터 이송,,
0,108000200,2020-09-07,확진자 접촉,True,9.1,2020-09-03,개포동,코이카생활치료센터 이송,,
0,108000199,2020-09-07,확진자 접촉,True,8.20,2020-09-02,세곡동,9.3 코이카생활치료센터 이송,,
0,108000198,2020-09-07,미상,True,8.31,2020-09-02,압구정동,서울의료원 이송,,


In [7]:
route

,num,ord,dat,rgl,rgt,frm,exd,msk,ste,mob
0,108000207,1,2020-09-07,,,,,,,
0,108000206,1,2020-09-07,신사동압구정역 2번 출구,판매점,,- 9.1(화) 20:14,,완료,도보
1,108000206,2,2020-09-07,압구정동압구정역 6번 출구,판매점,,- 9.1(화) 20:28,,완료,도보
0,108000205,1,2020-09-07,신사동압구정역 2번 출구,판매점,,- 9.1(화) 20:14,,완료,도보
1,108000205,2,2020-09-07,신사동압구정역 6번 출구,판매점,,- 9.1(화) 20:28,,완료,도보
...,...,...,...,...,...,...,...,...,...,...
6,108000175,7,2020-09-07,논현동언주역 3번 출구,헬스장,,- 8.28(금) 19:30~21:30,,완료,도보
7,108000175,8,2020-09-07,논현동언주역 2번 출구,병의원,,- 8.29(토) 10:30,,완료,도보
8,108000175,9,2020-09-07,논현동언주역 2번 출구,약국,,- 8.29(토) 10:35,,완료,도보
9,108000175,10,2020-09-07,논현동학동역 4번 출구,마트,,- 8.29(토) 10:39,,완료,도보
